<a href="https://colab.research.google.com/github/Prakash-sk26/student/blob/main/stud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import sqlite3

# ---------- Database Setup ----------
conn = sqlite3.connect("students.db")
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS students (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        roll_no TEXT NOT NULL,
        course TEXT NOT NULL,
        gender TEXT NOT NULL,
        email TEXT NOT NULL,
        phone TEXT NOT NULL
    )
""")
conn.commit()

# ---------- Functions ----------
def submit_form():
    name = entry_name.get()
    roll_no = entry_roll.get()
    course = entry_course.get()
    gender = gender_var.get()
    email = entry_email.get()
    phone = entry_phone.get()

    if not (name and roll_no and course and gender and email and phone):
        messagebox.showwarning("Validation Error", "Please fill all the fields!")
        return

    cursor.execute("INSERT INTO students (name, roll_no, course, gender, email, phone) VALUES (?, ?, ?, ?, ?, ?)",
                   (name, roll_no, course, gender, email, phone))
    conn.commit()

    messagebox.showinfo("Success", "Student registered successfully!")
    clear_form()

def clear_form():
    entry_name.delete(0, tk.END)
    entry_roll.delete(0, tk.END)
    entry_course.delete(0, tk.END)
    gender_var.set("")
    entry_email.delete(0, tk.END)
    entry_phone.delete(0, tk.END)

def search_student():
    student_id = entry_search.get()
    if not student_id.isdigit():
        messagebox.showerror("Error", "Please enter a valid ID")
        return

    cursor.execute("SELECT * FROM students WHERE id = ?", (student_id,))
    student = cursor.fetchone()

    if student:
        entry_name.delete(0, tk.END)
        entry_roll.delete(0, tk.END)
        entry_course.delete(0, tk.END)
        gender_var.set("")
        entry_email.delete(0, tk.END)
        entry_phone.delete(0, tk.END)

        entry_name.insert(0, student[1])
        entry_roll.insert(0, student[2])
        entry_course.insert(0, student[3])
        gender_var.set(student[4])
        entry_email.insert(0, student[5])
        entry_phone.insert(0, student[6])

        messagebox.showinfo("Student Found", "Details loaded for editing.")
    else:
        messagebox.showwarning("Not Found", "No student found with this ID.")

def delete_student():
    student_id = entry_search.get()
    if not student_id.isdigit():
        messagebox.showerror("Error", "Please enter a valid ID")
        return

    cursor.execute("DELETE FROM students WHERE id = ?", (student_id,))
    conn.commit()

    if cursor.rowcount > 0:
        messagebox.showinfo("Deleted", "Student deleted successfully!")
        clear_form()
    else:
        messagebox.showwarning("Not Found", "No student found with this ID.")

def update_student():
    student_id = entry_search.get()
    if not student_id.isdigit():
        messagebox.showerror("Error", "Please enter a valid ID")
        return

    name = entry_name.get()
    roll_no = entry_roll.get()
    course = entry_course.get()
    gender = gender_var.get()
    email = entry_email.get()
    phone = entry_phone.get()

    if not (name and roll_no and course and gender and email and phone):
        messagebox.showwarning("Validation Error", "Please fill all the fields!")
        return

    cursor.execute("""
        UPDATE students
        SET name=?, roll_no=?, course=?, gender=?, email=?, phone=?
        WHERE id=?
    """, (name, roll_no, course, gender, email, phone, student_id))
    conn.commit()

    if cursor.rowcount > 0:
        messagebox.showinfo("Updated", "Student details updated successfully!")
    else:
        messagebox.showwarning("Not Found", "No student found with this ID.")

def view_all_students():
    view_win = tk.Toplevel(root)
    view_win.title("📋 All Students")
    view_win.geometry("800x400")
    view_win.configure(bg="white")

    # Table
    cols = ("ID", "Name", "Roll No", "Course", "Gender", "Email", "Phone")
    tree = ttk.Treeview(view_win, columns=cols, show="headings")
    for col in cols:
        tree.heading(col, text=col)
        tree.column(col, width=100, anchor="center")

    # Scrollbars
    vsb = ttk.Scrollbar(view_win, orient="vertical", command=tree.yview)
    hsb = ttk.Scrollbar(view_win, orient="horizontal", command=tree.xview)
    tree.configure(yscroll=vsb.set, xscroll=hsb.set)
    vsb.pack(side="right", fill="y")
    hsb.pack(side="bottom", fill="x")

    tree.pack(fill="both", expand=True)

    # Fetch data
    cursor.execute("SELECT * FROM students")
    for row in cursor.fetchall():
        tree.insert("", tk.END, values=row)

# ---------- GUI Setup ----------
root = tk.Tk()
root.title("🎓 Student Management System")
root.geometry("500x800")
root.configure(bg="#f4f4f4")
root.resizable(False, False)

title_label = tk.Label(root, text="🎓 Student Management System", font=("Arial", 18, "bold"), bg="#1E90FF", fg="white", pady=10)
title_label.pack(fill="x")

# ----- Registration Frame -----
frame_reg = tk.LabelFrame(root, text="Student Registration", font=("Arial", 12, "bold"), bg="white", fg="#333", padx=10, pady=10)
frame_reg.pack(fill="x", padx=15, pady=10)

tk.Label(frame_reg, text="Name:", font=("Arial", 10, "bold"), bg="white").pack(anchor="w")
entry_name = tk.Entry(frame_reg, font=("Arial", 10))
entry_name.pack(fill="x", pady=3)

tk.Label(frame_reg, text="Roll No:", font=("Arial", 10, "bold"), bg="white").pack(anchor="w")
entry_roll = tk.Entry(frame_reg, font=("Arial", 10))
entry_roll.pack(fill="x", pady=3)

tk.Label(frame_reg, text="Course:", font=("Arial", 10, "bold"), bg="white").pack(anchor="w")
entry_course = tk.Entry(frame_reg, font=("Arial", 10))
entry_course.pack(fill="x", pady=3)

tk.Label(frame_reg, text="Gender:", font=("Arial", 10, "bold"), bg="white").pack(anchor="w")
gender_var = tk.StringVar()
tk.Radiobutton(frame_reg, text="Male", variable=gender_var, value="Male", bg="white").pack(anchor="w")
tk.Radiobutton(frame_reg, text="Female", variable=gender_var, value="Female", bg="white").pack(anchor="w")
tk.Radiobutton(frame_reg, text="Other", variable=gender_var, value="Other", bg="white").pack(anchor="w")

tk.Label(frame_reg, text="Email:", font=("Arial", 10, "bold"), bg="white").pack(anchor="w")
entry_email = tk.Entry(frame_reg, font=("Arial", 10))
entry_email.pack(fill="x", pady=3)

tk.Label(frame_reg, text="Phone:", font=("Arial", 10, "bold"), bg="white").pack(anchor="w")
entry_phone = tk.Entry(frame_reg, font=("Arial", 10))
entry_phone.pack(fill="x", pady=3)

tk.Button(frame_reg, text="Register Student", command=submit_form, bg="#1E90FF", fg="white", font=("Arial", 10, "bold")).pack(pady=10)

# ----- Search/Update/Delete Frame -----
frame_ops = tk.LabelFrame(root, text="Search / Update / Delete", font=("Arial", 12, "bold"), bg="white", fg="#333", padx=10, pady=10)
frame_ops.pack(fill="x", padx=15, pady=10)

tk.Label(frame_ops, text="Enter Student ID:", font=("Arial", 10, "bold"), bg="white").pack(anchor="w")
entry_search = tk.Entry(frame_ops, font=("Arial", 10))
entry_search.pack(fill="x", pady=5)

btn_frame = tk.Frame(frame_ops, bg="white")
btn_frame.pack(pady=5)

tk.Button(btn_frame, text="Search", command=search_student, bg="#28a745", fg="white", font=("Arial", 10, "bold"), width=10).grid(row=0, column=0, padx=5)
tk.Button(btn_frame, text="Update", command=update_student, bg="#ffc107", fg="black", font=("Arial", 10, "bold"), width=10).grid(row=0, column=1, padx=5)
tk.Button(btn_frame, text="Delete", command=delete_student, bg="#dc3545", fg="white", font=("Arial", 10, "bold"), width=10).grid(row=0, column=2, padx=5)

# ----- View All Students -----
tk.Button(root, text="📋 View All Students", command=view_all_students, bg="#17a2b8", fg="white", font=("Arial", 12, "bold")).pack(pady=15)

root.mainloop()
conn.close()